# 雪球期权
![grid](../note_pic/snowball.png)

## 到期收益分析
| 情景 | 具体分析| 合约实际结束时间   |       投资者收益       |  是否获得正收益  |
|:--:|:---:|:---:|:-----------------:|:---------:|
| 敲出 |从第三个月开始，10个敲出观察日中任何一天标的资产大于或等于沪深300初始价格的103%|T日合约提前终止| 25% * T/365 * 本金  |     是     |
| 未敲入且未敲出 |标的价格每日都不低于初始价格的80%，且每个敲出观察日都小于初始价格的103%|合约持有到期|25% * 360/365 * 本金|是(收益最高)|
|敲入且未敲出|任一交易日跌破初始价格的80%，且10个敲出观察日都小于初始价格的103%，<mark>标的资产的期末价格小于期初价格<mark>|合约持有到期|(期末价格/期初价格 - 1) * 本金|否(发生亏损)|
|敲入且未敲出|任一交易日跌破初始价格的80%，且10个敲出观察日都小于初始价格的103%，<mark>标的资产的期末价格处于期初价格和敲出价格之间</mark>|合约持有到期|0|否(保本)|  

1. 触发敲出，产品提前终止
根据图中案例产品，从第三个月开始，***每月***观察敲入 + ***每日***观察敲出。当标的资产价格大于敲出价格，投资者将会获得25%的年化收益率，产品提前赎回。  
产品在存续期间可能发生两种情况：在触发敲出前***从未发生敲入***；或者是***曾经发生过敲入***。在这两种情况下，***收益均为25% * T/365 * 本金，并全额收回本金***   
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.06.10.png)

2. 从未发生敲入或敲出
存续期间从未发生敲入或敲出，直至产品到期。收益为***收益均为25% * 360/365 * 本金，并全额收回本金***  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.09.52.png)

3. 敲入且未敲出
此时，投资者的收益由标的资产的到期价格决定。若标的资产的到期价格高于期初价格但是低于敲出价格，则收回本金且没有任何额外收益。  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.14.24.png)

若标的资产的到期价格低于起初价格，此时不仅不能获得收益，还需要承担资产在存续期间的跌幅所对应的名义本金价值，亏损金额为(期末价格/期初价格 - 1) * 名义本金，这也是雪球产品本金发生亏损的唯一情况  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.17.02.png)

In [1]:
import numpy as np
from scipy import interpolate
import scipy as sp
import time

In [2]:
class parameters():
    """parameters to used for pricing snowball option using monte carlo"""

    def __init__(self):
        """initialize parameters"""

        self.S = 1  # underlying spot
        self.K = 1  # strike
        self.KI_Barrier = 0.75  # down in barrier of put
        self.KO_Barrier = 1.03  # autocall barrier
        self.KO_Coupon = 0.22  # autocall coupon (p.a.)
        self.Bonus_Coupon = 0.22  # bonus coupon (p.a.)
        self.r = 0.03  # risk-free interest rate
        self.div = 0.01  # dividend rate
        self.repo = 0.08  # repo rate
        self.T = 2  # time to maturity in years
        self.v = 0.12  # volatility
        self.N = 252 * self.T  # number of discrete time points for whole tenor
        self.n = int(self.N / (self.T * 12))  # number of dicrete time point for each month
        self.M = int(self.T * 12)  # number of months
        self.dt = self.T / self.N  # delta t
        self.simulations = 30000
        self.J = 900  # number of steps of uly in the scheme
        self.lb = 0  # lower bound of domain of uly
        self.ub = 1.5  # upper bound of domain of uly
        self.dS = (self.ub - self.lb) / self.J  # delta S

    def print_parameters(self):
        """print parameters"""

        print("---------------------------------------------")
        print("Pricing a Snowball option using PDE")
        print("---------------------------------------------")
        print("Parameters of Snowball Option Pricer:")
        print("---------------------------------------------")
        print("Underlying Asset Price = ", self.S)
        print("Strike = ", self.K)
        print("Knock-in Barrier = ", self.KI_Barrier)
        print("Autocall Barrier = ", self.KO_Barrier)
        print("Autocall Coupon = ", self.KO_Coupon)
        print("Bonus Coupon = ", self.Bonus_Coupon)
        print("Risk-Free Rate =", self.r)
        print("Dividend Rate =", self.div)
        print("Repo Rate =", self.repo)
        print("Years Until Expiration = ", self.T)
        print("Volatility = ", self.v)
        print("Discrete time points =", self.N)
        print("Time-Step = ", self.dt)
        print("Underlyign domain = [", self.lb, ",", self.ub, "]")
        print("Discrete underlying points =", self.J)
        print("Underlying-Step = ", self.dS)
        print("---------------------------------------------")


In [3]:
class snowball_mc(parameters):

    def __init__(self):
        parameters.__init__(self)
        self.price_trajectories = []  # prices of all simulation paths
        self.option_price = np.nan  # option price using MC
        self.delta = np.nan
        self.gamma = np.nan
        self.vega = np.nan

    def compute_price(self):

        # reset trajectory
        self.price_trajectories = []

        # start simulation
        for i in range(self.simulations):
            e = sp.random.normal(0, 1, self.N)

            stockprices = np.cumprod(np.exp((self.r - self.div - self.repo -
                                             0.5 * self.v ** 2) * self.dt \
                                            + self.v * np.sqrt(self.dt) * e)) * self.S

            # check if Knockout, monthly observation
            n = int(1.0 / self.dt / 12.0)  # number of time points in every month
            s = slice(n - 1, self.N, n)
            stockprices_slice = stockprices[s]
            if stockprices_slice.max() >= self.KO_Barrier:
                idx = np.argmax(stockprices_slice >= self.KO_Barrier)
                time_to_KO = (idx + 1) / 12.0
                pv = (self.KO_Coupon * time_to_KO + 1) * np.exp(-self.r * time_to_KO)
                self.price_trajectories.append(pv)
                continue

            # if no KO, bonus coupon or down in put
            indicator_KI = stockprices.min() <= self.KI_Barrier
            indicator_S0_up = stockprices[-1] >= stockprices[0]
            pv = ((1 - indicator_KI) * (self.Bonus_Coupon * self.T + 1) \
                  + indicator_S0_up * indicator_KI * (stockprices[-1] - self.K + 1)) * np.exp(-self.r * self.T)
            self.price_trajectories.append(pv)

        self.option_price = np.sum(self.price_trajectories) / self.simulations

    def compute_greeks(self):
        """"compute greeks of snowball option"""

        epsilon = 0.01
        S0 = self.S

        # price with S = S0 * (1 - epsilon)
        self.S = S0 * (1 - epsilon)
        self.compute_price()
        P1 = self.option_price

        # price with S = S0 * (1 + epsilon)
        self.S = S0 * (1 + epsilon)
        self.compute_price()
        P2 = self.option_price

        # price with S = S0 and vol = vol + epsilon
        self.S = S0
        self.v = self.v + epsilon
        self.compute_price()
        P3 = self.option_price

        # back to original and price option price
        self.v = self.v - epsilon
        self.compute_price()
        P0 = self.option_price

        self.delta = (P2 - P1) / (2 * S0 * epsilon)
        self.gamma = (P1 + P2 - 2 * P0) / (S0 ** 2 * epsilon ** 2)
        self.vega = (P3 - P0) / epsilon

In [4]:
class snowball_pde(parameters):

    def __init__(self):
        parameters.__init__(self)
        self.Mat_Inv = self.__getInvMat()  # inverse matrix used in backwardation
        self.option_price = np.nan
        self.__V = np.zeros((self.J + 1, self.N + 1))  # backwardation grid
        self.delta = np.nan
        self.gamma = np.nan
        self.vega = np.nan

    """" 3 helper function to calculate inverse matrix needed"""

    def __a0(self, x):
        return 0.5 * self.dt * ((self.r - self.div - self.repo) * x - self.v ** 2 * x ** 2)

    def __a1(self, x):
        return 1 + self.r * self.dt + self.v ** 2 * x ** 2 * self.dt

    def __a2(self, x):
        return 0.5 * self.dt * (-(self.r - self.div - self.repo) * x - self.v ** 2 * x ** 2)

    def __getInvMat(self):
        """Calculating Inverse Matrix"""

        # first line
        V = np.zeros((self.J + 1, self.J + 1))
        V[0, 0] = 1.0 / (1 - self.r * self.dt)

        # lines between
        for i in range(1, self.J):
            V[i, i - 1] = self.__a0(i)
            V[i, i] = self.__a1(i)
            V[i, i + 1] = self.__a2(i)

        # last line
        V[self.J, self.J - 1] = self.__a0(self.J) - self.__a2(self.J)
        V[self.J, self.J] = self.__a1(self.J) + 2 * self.__a2(self.J)

        return np.matrix(V).I

    def __interpolate_price(self, y, s):

        x = [self.lb + self.dS * i for i in range(self.J + 1)]
        f = interpolate.interp1d(x, y, kind='cubic')

        return float(f(s))

    def __compute_autocall(self):
        """present value of autocall coupon if KO"""

        # initialize payoff at maturity
        V_terminal = np.zeros((self.J + 1, 1))
        V_terminal[slice(int((self.KO_Barrier - self.lb) / self.dS), \
                         self.J + 1, 1)] = self.KO_Coupon + 1
        V_matrix = np.zeros((self.J + 1, self.N + 1))
        V_matrix[:, -1] = V_terminal.reshape((self.J + 1,))

        # backwardation
        for i in range(self.M):
            for j in range(self.n):
                idx = i * self.n + j
                V_matrix[:, self.N - idx - 1] = (self.Mat_Inv * \
                                                 V_matrix[:, self.N - idx].reshape((self.J + 1, 1))).reshape(
                    (self.J + 1,))

            # pay coupon if KO at the end of each month
            KO_Coupon_temp = self.KO_Coupon * (self.T * 12 - i - 1) / 12
            if i != self.M - 1:
                V_matrix[:, self.N - idx - 1] \
                    [slice(int((self.KO_Barrier - self.lb) / self.dS), self.J + 1, 1)] = KO_Coupon_temp + 1

        self.__V = self.__V + V_matrix

    def __compute_bonus(self):
        """present value of bonus coupon if not KO and not KI"""

        # initialize payoff at maturity
        V_terminal = np.zeros((self.J + 1, 1))
        V_terminal[slice(int((self.KI_Barrier - self.lb) / self.dS), \
                         int((self.KO_Barrier - self.lb) / self.dS), 1)] = self.Bonus_Coupon + 1
        V_matrix = np.zeros((self.J + 1, self.N + 1))
        V_matrix[:, -1] = V_terminal.reshape((self.J + 1,))

        # backwardation
        for i in range(self.M):
            for j in range(self.n):
                idx = i * self.n + j
                V_matrix[:, self.N - idx - 1] = (self.Mat_Inv * \
                                                 V_matrix[:, self.N - idx].reshape((self.J + 1, 1))).reshape(
                    (self.J + 1,))

                # no bonus coupon if knock in, observed daily
                V_matrix[:, self.N - idx - 1][slice(0, \
                                                    int((self.KI_Barrier - self.lb) / self.dS), 1)] = 0

            # no bonus coupon if knock out, observed monthly
            if i != self.M - 1:
                V_matrix[:, self.N - idx - 1] \
                    [slice(int((self.KO_Barrier - self.lb) / self.dS), self.J + 1, 1)] = 0

        self.__V = self.__V + V_matrix

    def __compute_put_UO(self):
        """value of put up and out"""

        # initialize payoff at maturity
        V_terminal = np.array([-1 + max(self.K - i * self.dS, 0) for i in range(self.J + 1)]).reshape((self.J + 1, 1))
        V_terminal[slice(int((self.KO_Barrier - self.lb) / self.dS) + 0, self.J + 1, 1)] = 0
        V_matrix = np.zeros((self.J + 1, self.N + 1))
        V_matrix[:, -1] = V_terminal.reshape((self.J + 1,))

        # backwardation
        for i in range(self.M):
            for j in range(self.n):
                idx = i * self.n + j
                V_matrix[:, self.N - idx - 1] = (self.Mat_Inv * \
                                                 V_matrix[:, self.N - idx].reshape((self.J + 1, 1))).reshape(
                    (self.J + 1,))

            # nothing if Knock out, observed monthly
            if i != self.M - 1:
                V_matrix[:, self.N - idx - 1] \
                    [slice(int((self.KO_Barrier - self.lb) / self.dS) + 0, self.J + 1, 1)] = 0

        self.__V = self.__V - V_matrix

    def __compute_put_UO_DO(self):
        """value of put down&out and up&out"""

        # initialize payoff at maturity
        V_terminal = np.array([-1 + max(self.K - i * self.dS, 0) for i in range(self.J + 1)]).reshape((self.J + 1, 1))
        V_terminal[slice(0, int((self.KI_Barrier - self.lb) / self.dS), 1)] = 0
        V_terminal[slice(int((self.KO_Barrier - self.lb) / self.dS) + 1, self.J + 1, 1)] = 0
        V_matrix = np.zeros((self.J + 1, self.N + 1))
        V_matrix[:, -1] = V_terminal.reshape((self.J + 1,))

        # backwardation
        for i in range(self.M):
            for j in range(self.n):
                idx = i * self.n + j
                V_matrix[:, self.N - idx - 1] = (self.Mat_Inv * \
                                                 V_matrix[:, self.N - idx].reshape((self.J + 1, 1))).reshape(
                    (self.J + 1,))

                # nothing if knock in, observed daily
                V_matrix[:, self.N - idx - 1] \
                    [slice(0, int((self.KI_Barrier - self.lb) / self.dS), 1)] = 0

            # nothing if knock out, observed monthly
            if i != self.M - 1:
                V_matrix[:, self.N - idx - 1] \
                    [slice(int((self.KO_Barrier - self.lb) / self.dS) + 1, self.J + 1, 1)] = 0

        self.__V = self.__V + V_matrix

    def compute_price(self):
        """compute the price of snowball option"""

        # reset inverse Matrix in case vol has been changed
        self.Mat_Inv = self.__getInvMat()

        self.__V = np.zeros((self.J + 1, self.N + 1))
        self.__compute_autocall()
        self.__compute_bonus()
        self.__compute_put_UO()
        self.__compute_put_UO_DO()

        self.option_price = self.__interpolate_price(self.__V[:, 0], self.S)

    def compute_greeks(self):
        """"compute greeks of snowball option"""

        epsilon = 0.01
        self.v = self.v + epsilon
        self.compute_price()
        P3 = self.option_price

        # back to original and price
        self.v = self.v - epsilon
        self.compute_price()
        P0 = self.option_price
        P1 = self.__interpolate_price(self.__V[:, 0], self.S * (1 - epsilon))
        P2 = self.__interpolate_price(self.__V[:, 0], self.S * (1 + epsilon))

        self.delta = (P2 - P1) / (2 * self.S * epsilon)
        self.gamma = (P1 + P2 - 2 * P0) / (self.S ** 2 * epsilon ** 2)
        self.vega = (P3 - P0) / epsilon


In [5]:
pde = snowball_pde()
pde.print_parameters()
tic = time.time()
print("Starting calculating......", end="")
pde.compute_price()
print("Done.")
print("Option price = ", pde.option_price)
print("Running time = ", time.time() - tic, "s")
print("---------------------------------------------")

tic = time.time()
print("Calculating Greeks.....", end="")
pde.compute_greeks()
print("Done.")
print("Option delta = ", pde.delta)
print("Option gamma = ", pde.gamma)
print("Option vega = ", pde.vega)
print("Running time = ", time.time() - tic, "s")
print("---------------------------------------------")

---------------------------------------------
Pricing a Snowball option using PDE
---------------------------------------------
Parameters of Snowball Option Pricer:
---------------------------------------------
Underlying Asset Price =  1
Strike =  1
Knock-in Barrier =  0.75
Autocall Barrier =  1.03
Autocall Coupon =  0.22
Bonus Coupon =  0.22
Risk-Free Rate = 0.03
Dividend Rate = 0.01
Repo Rate = 0.08
Years Until Expiration =  2
Volatility =  0.12
Discrete time points = 504
Time-Step =  0.003968253968253968
Underlyign domain = [ 0 , 1.5 ]
Discrete underlying points = 900
Underlying-Step =  0.0016666666666666668
---------------------------------------------
Starting calculating......Done.
Option price =  1.006005435345123
Running time =  0.48920178413391113 s
---------------------------------------------
Calculating Greeks.....Done.
Option delta =  0.442250489118301
Option gamma =  -14.239691223640882
Option vega =  -1.0760392003905217
Running time =  0.9662361145019531 s
------------